For the known pathogenic, codis, and medically relevant genes, ask
1) How many there are in the catalog
2) How many are in the benchmark
3) How many have >=5bp HG002 variants in the benchmark

In [ ]:
import pyranges

In [1]:
import pandas as pd

catalog = pd.read_csv("/Users/english/code/adotto/regions/adotto_TRregions_v1.1.bed", sep='\t')
catalog.set_index(['chr', 'start', 'end'], inplace=True)
benchmark = pd.read_csv("/Users/english/code/adotto/benchmark/GIABTR_benchmark.6.26/GIABTR.HG002.benchmark.regions.bed.gz",
                        sep='\t', names=['chr', 'start', 'end', 'tier', 'repl', 'var_state', 'entropy', 'mat_ad', 'pat_ad'])
benchmark.set_index(['chr', 'start', 'end'], inplace=True)

In [3]:
data = benchmark.join(catalog)

In [38]:
cat_in_cmg = pd.read_csv("catalog_hitting_mrg.bed", sep='\t',
                    names=['chr', 'start', 'end', 'gchr', 'gstart', 'gend', 'gene', 'ovl']).set_index(['chr', 'start', 'end'])

In [62]:
patho = catalog[catalog['patho'] != '.']
codis = catalog[catalog['codis'] != '.']
bench_patho = data[data['patho'] != '.']
bench_codis = data[data['codis'] != '.']
gene = pd.read_csv("GRCh38_mrg_full_gene.bed", sep='\t', names=['chr', 'start', 'end', 'gene'])
has_hit_cmg = cat_in_cmg[cat_in_cmg['gchr'] != '.']
bench_gene = data[data.index.isin(has_hit_cmg.index)]
                   

In [63]:
def make_summary(d_bare, d_bench):
    """
    Okay, we gotta turn this into a table. Its too many numbers.
    Columns:
    - N : Total number of items there are (*on MRG as it'll have 1->N Gene to TRs
    - benchmark : Total number of items in benchmark
    - Tier1 count :
    - HG002 >=5bp Variant :
    - Non-HG002 >=5bp Variant :
    """
    ret = d_bench['tier'].value_counts()
    ret['N'] = len(d_bare)
    ret['benchmark'] = len(d_bench)
    ret['HG002 >=5bp'] = (d_bench['var_state'].apply(lambda x: (x & 0x1) != 0)).sum()
    ret['Other >=5bp'] = (d_bench['var_state'].apply(lambda x: (x & 0x4) != 0)).sum()
    return ret
    
    

In [64]:
p_summary = make_summary(patho, bench_patho)
p_summary.name = "Pathogenic"
c_summary = make_summary(codis, bench_codis)
c_summary.name = "CODIS"
g_summary = make_summary(has_hit_cmg, bench_gene)
g_summary.name = "MRG"

summary = pd.concat([p_summary, c_summary, g_summary], axis=1).T

In [66]:
summary[["N", "benchmark", "Tier1", "Tier2", "HG002 >=5bp", "Other >=5bp"]]

,N,benchmark,Tier1,Tier2,HG002 >=5bp,Other >=5bp
Pathogenic,62,50,42,8,24,40
CODIS,51,44,26,18,25,44
MRG,299633,289964,278225,11739,17201,49219


In [44]:
has_hit_cmg = cat_in_cmg[cat_in_cmg['gchr'] != '.']

In [45]:
print("Patho:", (data['patho'] != '.').sum())
print("CODIS:", (data['codis'] != '.').sum())
print("CMG:", data.index.isin(has_hit_cmg.index).sum())

Patho: 50
CODIS: 44
CMG: 289964


In [46]:
print(50 / 62)
print(44 / 51)
print(289964 / 299633)

0.8064516129032258
0.8627450980392157
0.9677305236739612


In [21]:
data.columns

Index(['tier', 'repl', 'var_state', 'entropy', 'mat_ad', 'pat_ad', 'ovl_flag',
       'up_buff', 'dn_buff', 'hom_pct', 'n_filtered', 'n_annos',
       'n_subregions', 'mu_purity', 'pct_annotated', 'interspersed', 'patho',
       'codis', 'gene_flag', 'biotype', 'annos'],
      dtype='object')

In [22]:
print("Patho:", data[data['patho'] != '.']['tier'].value_counts())


Patho: Tier1    42
Tier2     8
Name: tier, dtype: int64


In [23]:
print("CODIS:", data[data['codis'] != '.']['tier'].value_counts())


CODIS: Tier1    26
Tier2    18
Name: tier, dtype: int64


In [47]:
print("CMG:", data[data.index.isin(has_hit_cmg.index)]['tier'].value_counts())

CMG: Tier1    278225
Tier2     11739
Name: tier, dtype: int64


In [48]:
278225 / (278225 + 11739)

0.9595156640134638

In [16]:
data[data['patho'] != '.']['var_state'].value_counts()

13    15
12    11
15     9
8      8
14     5
10     2
Name: var_state, dtype: int64

In [17]:
data[data['codis'] != '.']['var_state'].value_counts()

14    14
15    13
13    11
12     5
5      1
Name: var_state, dtype: int64

In [18]:
cat_view = data[data.index.isin(cat_in_cmg.index)]

In [19]:
cat_view['var_state'].value_counts()

8     115442
0      83822
10     40516
14     15774
12     13520
15      9720
13      6610
4       2569
5        754
2        159
11        26
9         17
6          7
1          4
7          1
Name: var_state, dtype: int64

Okay, we gotta turn this into a table. Its too many numbers.

Columns:
- Site : Pathogenic, CODIS, MRG
- N : Total number of items there are (*on MRG as it'll have 1->N Gene to TRs
- benchmark : Total number of items in benchmark
- Tier1 count :
- HG002 >=5bp Variant :
- Non-HG002 >=5bp Variant :